In [5]:
import torch
from torch import nn
import torch.onnx
import tensorflow as tf 
from onnx_tf.backend import prepare
import onnx

In [6]:
class ResBlock(nn.Module):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        
        self.resblock = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.InstanceNorm2d(channels, affine=True),
            nn.ReLU(),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.InstanceNorm2d(channels, affine=True),
        )
        
    def forward(self, x):
        out = self.resblock(x)
        return out + x

class MicroResNet(nn.Module):
    def __init__(self):
        super(MicroResNet, self).__init__()
        
        self.downsampler = nn.Sequential(
            nn.Conv2d(3, 8, kernel_size=9, padding=4),
            nn.InstanceNorm2d(8, affine=True),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3, padding=1, stride=2),
            nn.InstanceNorm2d(16, affine=True),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1, stride=4),
            nn.InstanceNorm2d(32, affine=True),
            nn.ReLU()
        )
        
        self.residual = nn.Sequential(
            ResBlock(32),
            ResBlock(32)
        )
        
        self.segmentator = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.InstanceNorm2d(16, affine=True),
            nn.ReLU(),
            nn.Conv2d(16, 1, kernel_size=9, padding=4),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        out = self.downsampler(x)
        for i in range(2): # recursively use the same simple block 2 times
            out = self.residual(out)
        out = self.segmentator(out)
        return out

In [7]:
model = MicroResNet()

checkpoint = torch.load('saliency_model_v4.pt')
model.load_state_dict(checkpoint)
model.eval()

MicroResNet(
  (downsampler): Sequential(
    (0): Conv2d(3, 8, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (1): InstanceNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (2): ReLU()
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (5): ReLU()
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(4, 4), padding=(1, 1))
    (7): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (8): ReLU()
  )
  (residual): Sequential(
    (0): ResBlock(
      (resblock): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (2): ReLU()
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): InstanceNorm2d(32, eps=1e-05, momentum=0.1,

In [9]:
with torch.no_grad():
    x = torch.randn(1, 3, 240, 320, requires_grad=True)
    torch_out = torch.onnx._export(model, x, "saliency.onnx", export_params=True)

In [10]:
onnx_model = onnx.load("saliency.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("saliency.pb")  # export the model

/usr/local/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/site-packages/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op MeanVarianceNormalization in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))


In [11]:
g = tf.GraphDef()
g.ParseFromString(open("saliency.pb", "rb").read())

# g.node
[n for n in g.node if n.name == '0']

[name: "0"
 op: "Placeholder"
 attr {
   key: "dtype"
   value {
     type: DT_FLOAT
   }
 }
 attr {
   key: "shape"
   value {
     shape {
       dim {
         size: 1
       }
       dim {
         size: 3
       }
       dim {
         size: 240
       }
       dim {
         size: 320
       }
     }
   }
 }]

In [4]:
# Tensorflow to Tensorflow Lite      IMPORTANT --input_arrays=0 --output_arrays=Sigmoid
!tflite_convert --output_file=saliency.tflite --graph_def_file=saliency.pb --input_arrays=0 --output_arrays=Sigmoid --inference_type=FLOAT


2019-02-07 20:00:04.600569: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-02-07 20:00:04.681223: I tensorflow/core/grappler/devices.cc:53] Number of eligible GPUs (core count >= 8): 0 (Note: TensorFlow was not compiled with CUDA support)
2019-02-07 20:00:04.681312: I tensorflow/core/grappler/clusters/single_machine.cc:359] Starting new session
2019-02-07 20:00:04.770914: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:586] Optimization results for grappler item: graph_to_optimize
2019-02-07 20:00:04.770935: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:588]   model_pruner: Graph size after: 434 nodes (-77), 501 edges (-25), time = 4.166ms.
2019-02-07 20:00:04.770941: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:588]   function_optimizer: Graph size after: 434 nodes (0), 501 edges (0), time = 1.052ms.
2019-02-07 20:00:04.770945: I tensorflow/core/grappler